### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [100]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

In [166]:
url = 'https://www.booking.com/searchresults.ko.html?aid=376440;label=bdot-SIcScZhJX6z_%2AYtUYg62hwS267777897793%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-325272469656%3Alp1009871%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo;sid=febadbcd7e51aee26acbfeeea6e8299e;city=-714191;from_idr=1;srpvid=7a6d467b344501c5&;dr_ps=IDR;ilp=1;d_dcp=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
driver = webdriver.Chrome(path)
driver.get(url)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [167]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="0932387b8ab6276477a452a337a8afc3", element="df32c89a-a76e-4cbe-9bd7-dabac9d80417")>

In [163]:
# 전체 페이지 개수 확인
pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
last_pages = ''

for idx, page in enumerate(pages):
    print(page.text)
    if idx == len(pages)-1: # 마지막 페이지
        last_pages = page.text


1
2
3
4
5
12


In [174]:
def get_curr():
    driver.implicitly_wait(2)
    time.sleep(2+random.uniform(0,1))

    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    
    return int(curr_page.text)

In [175]:
# 스크래핑 페이지 범위
while get_curr() <= int(last_pages):
    # scraping
    data_scraping()
    
    if get_curr() == int(last_pages):
        break
        
    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    print(curr_page.text,"페이지 완료")

    move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
    move_next.send_keys('\n')
        
    driver.implicitly_wait(2)
    time.sleep(2+random.uniform(0,1))

나비스 호텔 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
하모니 리조트 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
UTS Village ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
바다위에펜션 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
제주 갤럭시 호텔 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
제주 오라 스테이 호텔 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
트윈스호텔 ['3', '1', '3', '4', '3', ' ', '2', ' ', '3', '3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '3', '3', '3', '3', '3', ' ']
호텔 스카이파크 제주 1 ['3', '1', '3', '4', '3', ' ', '2', ' ', 

In [171]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        except Exception as NoSuchElementException:
            stars.append(' ')
    
    return stars

In [177]:
def data_scraping():
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')

    stars = []
    hotel_names = []
    
    stars = get_stars(listed)
    
    for i in range (1, len(listed)):
        # 중간 베너 제외
        if i == 11:
            continue
            
        hotel_name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
        hotel_names.append(hotel_name)
        
        print(hotel_name.text, stars[i])
        
data_scraping()

나비스 호텔 1
하모니 리조트 3
UTS Village 4
바다위에펜션 3
제주 갤럭시 호텔  
제주 오라 스테이 호텔 2
트윈스호텔  
호텔 스카이파크 제주 1 3
우도 사랑채 민박 3
W호텔 탑동  
예스준 게스트하우스  
고고애월  
제주 오션 패밀리 호텔  
제주도 하숙집  
제주 디셈버 호텔  
소근소근  
바오젠 게스트하우스  
협재 엔게디 하우스 3
백록담펜션 3
이쁜새 팬션 3
아침하늘펜션 3
비자림미담 독채펜션 3
선양 빌리지 돔 카라반  


IndexError: list index out of range

---

In [1]:
# 한글 설정
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

f_path = "/Users/seyoung/Library/Fonts/AppleGothic.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)